In [65]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from gensim.models import Word2Vec

In [66]:
data = pd.read_csv('george_wassouf_lyrics_dataset.csv')
data.head()

,Track Title,Track Lyrics,Release year
0,Aah Habaieb - آه حبايب,اه حبايب بس فين الحب غايب\nوليه يا قلبي ما لك...,1995
1,Aalem Albi El Shok - علم قلبي الشوق,\nيا ليلي يا ليلي يا ليل\n\nعلم قلبي الشوق الش...,2008
2,Addi Eli Kanou - أدي اللي كانوا,أدي اللي كانوا يا هوا أقرب ما لينا\nواللي ما ...,2008
3,Alb Al Asheq Daleiloh - قلب العاشق دليله,قلب العاشق دليله\nقلب العاشق دليله\n\nقلب العا...,2002
4,Al Forqah Saabah - الفرقة صعبة,مش قادر اصبرعغيابا\nكل ليله بجمع دموعي\nو بسهر...,2009


In [67]:
# Number of songs
no_of_songs = data.shape[0]
print(no_of_songs)
# Number of attributes
no_of_attributes = data.shape[1]
print(no_of_attributes)
# Attribute names
attrs = data.columns.tolist()
print(attrs)

78
3
['Track Title', 'Track Lyrics', 'Release year']


In [68]:
track_lyrics_df = data['Track Lyrics'].to_frame()
print(track_lyrics_df.head())

                                        Track Lyrics
0   اه حبايب بس فين الحب غايب\nوليه يا قلبي ما لك...
1  \nيا ليلي يا ليلي يا ليل\n\nعلم قلبي الشوق الش...
2   أدي اللي كانوا يا هوا أقرب ما لينا\nواللي ما ...
3  قلب العاشق دليله\nقلب العاشق دليله\n\nقلب العا...
4  مش قادر اصبرعغيابا\nكل ليله بجمع دموعي\nو بسهر...


In [69]:
data['Track Lyrics'] = data['Track Lyrics'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data.head()

,Track Title,Track Lyrics,Release year
0,Aah Habaieb - آه حبايب,اه حبايب بس فين الحب غايب\nوليه يا قلبي ما لك...,1995
1,Aalem Albi El Shok - علم قلبي الشوق,\nيا ليلي يا ليلي يا ليل\n\nعلم قلبي الشوق الش...,2008
2,Addi Eli Kanou - أدي اللي كانوا,أدي اللي كانوا يا هوا أقرب ما لينا\nواللي ما ...,2008
3,Alb Al Asheq Daleiloh - قلب العاشق دليله,قلب العاشق دليله\nقلب العاشق دليله\n\nقلب العا...,2002
4,Al Forqah Saabah - الفرقة صعبة,مش قادر اصبرعغيابا\nكل ليله بجمع دموعي\nو بسهر...,2009


In [70]:
data['Track Title'] = data['Track Title'].apply(lambda x: re.sub(r'[-A-Za-z]', '', x))
data.head()

,Track Title,Track Lyrics,Release year
0,آه حبايب,اه حبايب بس فين الحب غايب\nوليه يا قلبي ما لك...,1995
1,علم قلبي الشوق,\nيا ليلي يا ليلي يا ليل\n\nعلم قلبي الشوق الش...,2008
2,أدي اللي كانوا,أدي اللي كانوا يا هوا أقرب ما لينا\nواللي ما ...,2008
3,قلب العاشق دليله,قلب العاشق دليله\nقلب العاشق دليله\n\nقلب العا...,2002
4,الفرقة صعبة,مش قادر اصبرعغيابا\nكل ليله بجمع دموعي\nو بسهر...,2009


In [71]:
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_newline_chars(text):
    return text.replace('\n', ' ').replace('\r', '')

def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                                ّ    | # Tashdid
                               ~    | # Shada 
                                َ    | # Fatha
                                ً    | # Tanwin Fath
                                ُ    | # Damma
                                ٌ    | # Tanwin Damm
                                ِ    | # Kasra
                                ٍ    | # Tanwin Kasr
                                ْ    | # Sukun
                                ـ     # Tatwil/Kashida
                            """, re.VERBOSE)
    return re.sub(arabic_diacritics, '', text)

data['Track Lyrics'] = data['Track Lyrics'].apply(remove_punctuation)
data['Track Lyrics'] = data['Track Lyrics'].apply(remove_newline_chars)
data['Track Lyrics'] = data['Track Lyrics'].apply(remove_diacritics)
data['Track Title'] = data['Track Title'].apply(remove_punctuation)
data['Track Title'] = data['Track Title'].apply(remove_newline_chars)
data['Track Title'] = data['Track Title'].apply(remove_diacritics)

data.head()

,Track Title,Track Lyrics,Release year
0,آه حبايب,اه حبايب بس فين الحب غايب وليه يا قلبي ما لكش...,1995
1,علم قلبي الشوق,يا ليلي يا ليلي يا ليل علم قلبي الشوق الشوق ...,2008
2,أدي اللي كانوا,أدي اللي كانوا يا هوا أقرب ما لينا واللي ما ه...,2008
3,قلب العاشق دليله,قلب العاشق دليله قلب العاشق دليله قلب العاشق ...,2002
4,الفرقة صعبة,مش قادر اصبرعغيابا كل ليله بجمع دموعي و بسهر م...,2009


In [72]:
def normalize_arabic_text(text):
    text = re.sub(r'ي\b', 'ى', text)
    text = text.replace('أ', 'ا')
    text = text.replace('آ', 'ا')
    text = re.sub(r'\bإ', 'ا', text)
    text = re.sub(r'ة\b', 'ه', text)
    return text

data['Track Lyrics'] = data['Track Lyrics'].apply(normalize_arabic_text)
data['Track Title'] = data['Track Title'].apply(normalize_arabic_text)
data.head()

,Track Title,Track Lyrics,Release year
0,اه حبايب,اه حبايب بس فين الحب غايب وليه يا قلبى ما لكش...,1995
1,علم قلبى الشوق,يا ليلى يا ليلى يا ليل علم قلبى الشوق الشوق ...,2008
2,ادى اللى كانوا,ادى اللى كانوا يا هوا اقرب ما لينا واللى ما ه...,2008
3,قلب العاشق دليله,قلب العاشق دليله قلب العاشق دليله قلب العاشق ...,2002
4,الفرقه صعبه,مش قادر اصبرعغيابا كل ليله بجمع دموعى و بسهر م...,2009


In [74]:
stop_words = set(stopwords.words('arabic'))
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

data['Track Lyrics'] = data['Track Lyrics'].apply(remove_stopwords)
data['Track Title'] = data['Track Title'].apply(remove_stopwords)
data.head()

,Track Title,Track Lyrics,Release year
0,اه حبايب,اه حبايب فين الحب غايب وليه قلبى لكش نايب شافو...,1995
1,قلبى الشوق,ليلى ليلى ليل قلبى الشوق الشوق عينى السهر خدنى...,2008
2,ادى اللى كانوا,ادى اللى كانوا هوا اقرب لينا واللى هانوا هوا ف...,2008
3,قلب العاشق دليله,قلب العاشق دليله قلب العاشق دليله قلب العاشق د...,2002
4,الفرقه صعبه,مش قادر اصبرعغيابا ليله بجمع دموعى بسهر دمعات ...,2009
